In [44]:
import pandas as pd
import numpy as np

In [45]:
customers = pd.read_csv('./customers.csv')
customers.head()

,id,created
0,35410,2015-07-03 22:01:11
1,35417,2015-07-03 22:11:23
2,35412,2015-07-03 22:02:52
3,35413,2015-07-03 22:05:02
4,35424,2015-07-03 22:21:55


In [46]:
customers.shape

(25716, 2)

In [47]:
orders = pd.read_csv('./orders.csv')
orders.head()

,id,order_number,user_id,created
0,1709,36,344,2014-10-28 00:20:01
1,1406,7,608,2014-10-14 23:44:53
2,1716,6,2296,2014-10-28 17:47:07
3,1426,2,1225,2014-10-15 18:33:38
4,1415,6,797,2014-10-15 02:07:16


In [48]:
orders.shape

(27575, 4)

In [49]:
customers_orders = customers.merge(orders, left_on='id', right_on='user_id', suffixes=('_customer', '_order'))
customers_orders.head()

,id_customer,created_customer,id_order,order_number,user_id,created_order
0,35424,2015-07-03 22:21:55,27970,1,35424,2015-07-03 23:37:49
1,35399,2015-07-03 21:30:36,27940,1,35399,2015-07-03 22:17:24
2,35414,2015-07-03 22:09:04,27949,1,35414,2015-07-03 22:51:05
3,35452,2015-07-04 00:20:15,27988,1,35452,2015-07-04 00:30:13
4,35442,2015-07-03 23:33:17,27980,1,35442,2015-07-04 00:05:48


In [58]:
customers_orders['customer_cohort'] = pd.to_datetime(customers_orders.created_customer)
customers_orders['created_order'] = pd.to_datetime(customers_orders.created_order)
customers_orders.head()

,id_customer,created_customer,id_order,order_number,user_id,created_order,customer_cohort
0,35424,2015-07-03 22:21:55,27970,1,35424,2015-07-03 23:37:49,2015-07-03 22:21:55
1,35399,2015-07-03 21:30:36,27940,1,35399,2015-07-03 22:17:24,2015-07-03 21:30:36
2,35414,2015-07-03 22:09:04,27949,1,35414,2015-07-03 22:51:05,2015-07-03 22:09:04
3,35452,2015-07-04 00:20:15,27988,1,35452,2015-07-04 00:30:13,2015-07-04 00:20:15
4,35442,2015-07-03 23:33:17,27980,1,35442,2015-07-04 00:05:48,2015-07-03 23:33:17


### get the number of distinct customers that ordered in each cohort in each week

In [51]:
# groupby created column to get distinct # of customers that ordered in a week?
n_distinct_user_orders_by_week = customers_orders.groupby([pd.Grouper(key='customer_cohort', freq='W'), pd.Grouper(key='created_order', freq='W')]).agg({'user_id': pd.Series.nunique}).rename(columns={'user_id': 'n_distinct_customers'})
n_distinct_user_orders_by_week.head()

n_distinct_customers
customer_cohort created_order                      
2015-01-04      2015-01-04                       16
                2015-01-11                        8
                2015-01-18                       11
                2015-01-25                        9
                2015-02-01                        9

### Get the count of distinct customers that ordered for the first time by week

In [64]:
# first group by the customer_id
first_order_by_customer = orders.groupby('user_id').agg({'created': np.min}).rename(columns = {'created': 'first_order'}) # time of first order by user id
# convert the first order column to a datetime column
first_order_by_customer['first_order'] = pd.to_datetime(first_order_by_customer['first_order'])
first_order_by_customer = first_order_by_customer.reset_index()
# then, group by the first order week, count the number of user ids that fall within that week
count_first_order_by_week = first_order_by_customer.groupby(pd.Grouper(key = 'first_order', freq = 'W'))\
                                                   .agg({'user_id': pd.Series.nunique})\
                                                   .rename(columns={'user_id': 'first_order_count'})
count_first_order_by_week.head()

,first_order_count
first_order,
2014-05-18,2
2014-05-25,1
2014-06-01,1
2014-06-08,1
2014-06-15,0


In [62]:
first_order_by_customer.head()

,user_id,first_order
0,2,2014-05-16 00:13:50
1,3,2014-05-21 20:39:23
2,5,2014-05-18 01:36:30
3,10,2014-05-26 21:10:14
4,11,2014-06-03 22:25:15


In [68]:
customers_orders_w_first = customers_orders.merge(first_order_by_customer, left_on='id_customer', right_on='user_id', how='inner')
customers_orders_w_first.head()

,id_customer,created_customer,id_order,order_number,user_id_x,created_order,customer_cohort,user_id_y,first_order
0,35424,2015-07-03 22:21:55,27970,1,35424,2015-07-03 23:37:49,2015-07-03 22:21:55,35424,2015-07-03 23:37:49
1,35399,2015-07-03 21:30:36,27940,1,35399,2015-07-03 22:17:24,2015-07-03 21:30:36,35399,2015-07-03 22:17:24
2,35414,2015-07-03 22:09:04,27949,1,35414,2015-07-03 22:51:05,2015-07-03 22:09:04,35414,2015-07-03 22:51:05
3,35452,2015-07-04 00:20:15,27988,1,35452,2015-07-04 00:30:13,2015-07-04 00:20:15,35452,2015-07-04 00:30:13
4,35442,2015-07-03 23:33:17,27980,1,35442,2015-07-04 00:05:48,2015-07-03 23:33:17,35442,2015-07-04 00:05:48


Make a boolean column that marks whether an order is the first one?
when we group by the cohort and created order dates, count the number of trues

In [90]:
customers_orders_w_first['is_first'] = customers_orders_w_first.apply(lambda x: 1 if x['created_order'] == x['first_order'] else 0, axis = 1)
customers_orders_w_first.head()

,id_customer,created_customer,id_order,order_number,user_id_x,created_order,customer_cohort,user_id_y,first_order,is_first
0,35424,2015-07-03 22:21:55,27970,1,35424,2015-07-03 23:37:49,2015-07-03 22:21:55,35424,2015-07-03 23:37:49,1
1,35399,2015-07-03 21:30:36,27940,1,35399,2015-07-03 22:17:24,2015-07-03 21:30:36,35399,2015-07-03 22:17:24,1
2,35414,2015-07-03 22:09:04,27949,1,35414,2015-07-03 22:51:05,2015-07-03 22:09:04,35414,2015-07-03 22:51:05,1
3,35452,2015-07-04 00:20:15,27988,1,35452,2015-07-04 00:30:13,2015-07-04 00:20:15,35452,2015-07-04 00:30:13,1
4,35442,2015-07-03 23:33:17,27980,1,35442,2015-07-04 00:05:48,2015-07-03 23:33:17,35442,2015-07-04 00:05:48,1


In [91]:
customers_orders_w_first['is_first'].value_counts()

0    11659
1     5356
Name: is_first, dtype: int64

## get the sum of customer who made their first order, and the count of the unique customers who ordered in that week

# This is the shit that I need

In [95]:
cohorts_w_all_counts = customers_orders_w_first\
.groupby([pd.Grouper(key='customer_cohort', freq='W'), pd.Grouper(key='created_order', freq='W')])\
.agg({'id_customer': pd.Series.nunique, 'is_first': np.sum})\
.rename(columns = {'id_customer': 'unique_customers', 'is_first': 'n_first_orders'})

cohorts_w_all_counts.head()

unique_customers  n_first_orders
customer_cohort created_order                                  
2015-01-04      2015-01-04                   16              16
                2015-01-11                    8               4
                2015-01-18                   11               3
                2015-01-25                    9               1
                2015-02-01                    9               1

In [61]:
n_distinct_user_orders_by_week.merge(count_first_order_by_week, how='left', left_on='created_order', right_index=True)

# n_distinct_customers.merge(n_distinct_user_orders_by_week, how='left', left_on=['first_order'], right_on='created_dt', right_index=True)

n_distinct_customers  first_order_count
customer_cohort created_order                                         
2015-01-04      2015-01-04                       16                 58
                2015-01-11                        8                140
                2015-01-18                       11                305
                2015-01-25                        9                338
                2015-02-01                        9                255
...                                             ...                ...
2015-06-28      2015-07-05                       36                230
                2015-07-12                       24                136
2015-07-05      2015-07-05                      125                230
                2015-07-12                       48                136
2015-07-12      2015-07-12                       37                136

[406 rows x 2 columns]

### Join the customers table and the first order by customer table on the key 'id' in customers with the index of user_id in the first_order_by_customer table

In [87]:
customers_first_order = customers.join(first_order_by_customer, on='id')
customers_first_order.head()

,id,created,user_id,first_order
0,35410,2015-07-03 22:01:11,NaN,NaT
1,35417,2015-07-03 22:11:23,NaN,NaT
2,35412,2015-07-03 22:02:52,NaN,NaT
3,35413,2015-07-03 22:05:02,NaN,NaT
4,35424,2015-07-03 22:21:55,NaN,NaT


In [86]:
customers_first_order.shape

(0, 4)

For this exercise group the customers into week long (7 days) cohorts and then calculate how many distinct customers ordered within X days from their signup date, where X is a multiple of 7. Older cohorts will have more buckets: 0-6 days, 7-13 days, 14-20 days, etc.

### convert the 'created' column (the datetime that the customer was created/joined) to a datetime column, and call it customer cohort

In [13]:
customers_first_order['customer_cohort'] = pd.to_datetime(customers_first_order.created)
customers_first_order.head()

,id,created,first_order,customer_cohort
4,35424,2015-07-03 22:21:55,2015-07-03 23:37:49,2015-07-03 22:21:55
9,35399,2015-07-03 21:30:36,2015-07-03 22:17:24,2015-07-03 21:30:36
22,35414,2015-07-03 22:09:04,2015-07-03 22:51:05,2015-07-03 22:09:04
28,35452,2015-07-04 00:20:15,2015-07-04 00:30:13,2015-07-04 00:20:15
33,35442,2015-07-03 23:33:17,2015-07-04 00:05:48,2015-07-03 23:33:17


In [14]:
grouped_first_orders = customers_first_order.groupby([pd.Grouper(key='customer_cohort', freq='W'), pd.Grouper(key='first_order', freq='W')])\
    .agg({'id': pd.Series.nunique}).rename(columns={'id': 'n_first_orders'})
grouped_first_orders.head()
# grouped_first_orders.merge(n_distinct_user_orders_by_week, how='inner', right_index=True, left_on='first_order')


n_first_orders
customer_cohort first_order                
2015-01-04      2015-01-04               16
                2015-01-11                4
                2015-01-18                3
                2015-01-25                1
                2015-02-01                1

### group the customer_first_order table first by their customer_cohort with weekly frequency, then group them by the week of their first order. Aggregate the columns by the number of unique customer ids that fall within that group

In [15]:
n_distinct_customers = customers_first_order\
    .groupby([pd.Grouper(key='customer_cohort', freq='W'), pd.Grouper(key='first_order', freq='W')])\
    .agg({'id': pd.Series.nunique}).rename(columns={'id': 'n_first_orders'})
n_distinct_customers.head()

n_first_orders
customer_cohort first_order                
2015-01-04      2015-01-04               16
                2015-01-11                4
                2015-01-18                3
                2015-01-25                1
                2015-02-01                1

## n_distinct_customers is the number of distinct customers that ordered in a week. n_first_orders is the number of customers who ordered for the first time

In [22]:
n_distinct_customers.merge(n_distinct_user_orders_by_week, how='left', left_on=['first_order'], right_on='created_dt', right_index=True)

n_first_orders  n_distinct_customers
customer_cohort first_order                                      
2015-01-04      2015-01-04               16                   195
                2015-01-11                4                   351
                2015-01-18                3                   534
                2015-01-25                1                   630
                2015-02-01                1                   568
...                                     ...                   ...
2015-06-28      2015-07-05               17                   986
                2015-07-12               11                   534
2015-07-05      2015-07-05              125                   986
                2015-07-12               30                   534
2015-07-12      2015-07-12               37                   534

[360 rows x 2 columns]

In [11]:
n_distinct_customers.reset_index().set_index('first_order').join(n_distinct_user_orders_by_week, how='inner')

,customer_cohort,n_first_orders,n_distinct_customers
2015-01-04,2015-01-04,16,195
2015-01-11,2015-01-04,4,351
2015-01-11,2015-01-11,82,351
2015-01-18,2015-01-04,3,534
2015-01-18,2015-01-11,72,534
2015-01-18,2015-01-18,152,534
2015-01-25,2015-01-04,1,630
2015-01-25,2015-01-11,55,630
2015-01-25,2015-01-18,85,630
2015-01-25,2015-01-25,156,630


### get the number of unique customers within each customer cohort by summing across the first index.

In [11]:
sum_by_cohort = n_distinct_customers.sum(level=0).rename(columns={'n_unique_customers': 'cohort_unique_customers'})
sum_by_cohort.head()

,cohort_unique_customers
customer_cohort,
2015-01-04,32
2015-01-11,389
2015-01-18,394
2015-01-25,469
2015-02-01,234


### merge the sum_by_cohort and the n_distinct_customers dataframes on their first indices

In [12]:
n_distinct_customers = sum_by_cohort.merge(n_distinct_customers, left_index = True, right_index  = True)
n_distinct_customers.head()

cohort_unique_customers  n_unique_customers
customer_cohort first_order                                             
2015-01-04      2015-01-04                        32                  16
                2015-01-11                        32                   4
                2015-01-18                        32                   3
                2015-01-25                        32                   1
                2015-02-01                        32                   1

### Get the percentage of distinct customers that placed their first order within X days from their signup date, where X is a multiple of 7

In [25]:
n_distinct_customers['percent_customers'] = (n_distinct_customers.n_unique_customers / n_distinct_customers.cohort_unique_customers)
n_distinct_customers.head()

cohort_unique_customers  n_unique_customers  \
customer_cohort first_order                                                
2015-01-04      2015-01-04                        32                  16   
                2015-01-11                        32                   4   
                2015-01-18                        32                   3   
                2015-01-25                        32                   1   
                2015-02-01                        32                   1   

                             percent_customers  
customer_cohort first_order                     
2015-01-04      2015-01-04             0.50000  
                2015-01-11             0.12500  
                2015-01-18             0.09375  
                2015-01-25             0.03125  
                2015-02-01             0.03125

In [26]:
n_distinct_customers[:15]

cohort_unique_customers  n_unique_customers  \
customer_cohort first_order                                                
2015-01-04      2015-01-04                        32                  16   
                2015-01-11                        32                   4   
                2015-01-18                        32                   3   
                2015-01-25                        32                   1   
                2015-02-01                        32                   1   
                2015-02-08                        32                   1   
                2015-02-22                        32                   1   
                2015-03-08                        32                   1   
                2015-03-29                        32                   2   
                2015-05-10                        32                   1   
                2015-06-28                        32                   1   
2015-01-11      2015-01-11                       389                  82   
                2015-01-18                       389                  72   
                2015-01-25                       389                  55   
                2015-02-01                       389                  26   

                             percent_customers  
customer_cohort first_order                     
2015-01-04      2015-01-04            0.500000  
                2015-01-11            0.125000  
                2015-01-18            0.093750  
                2015-01-25            0.031250  
                2015-02-01            0.031250  
                2015-02-08            0.031250  
                2015-02-22            0.031250  
                2015-03-08            0.031250  
                2015-03-29            0.062500  
                2015-05-10            0.031250  
                2015-06-28            0.031250  
2015-01-11      2015-01-11            0.210797  
                2015-01-18            0.185090  
                2015-01-25            0.141388  
                2015-02-01            0.066838

In [53]:
n_distinct_customers.sort_index(ascending=False).reset_index()

,customer_cohort,first_order,cohort_unique_customers,n_unique_customers,percent_customers,formatted_percent_orders
0,2015-07-12,2015-07-12,37,37,1.000000,100% orderers (37)
1,2015-07-05,2015-07-12,155,30,0.193548,19% orderers (155)
2,2015-07-05,2015-07-05,155,125,0.806452,81% orderers (155)
3,2015-06-28,2015-07-12,164,11,0.067073,7% orderers (164)
4,2015-06-28,2015-07-05,164,17,0.103659,10% orderers (164)
5,2015-06-28,2015-06-28,164,136,0.829268,83% orderers (164)
6,2015-06-21,2015-07-12,197,6,0.030457,3% orderers (197)
7,2015-06-21,2015-07-05,197,12,0.060914,6% orderers (197)
8,2015-06-21,2015-06-28,197,43,0.218274,22% orderers (197)
9,2015-06-21,2015-06-21,197,136,0.690355,69% orderers (197)


### need to format a column nicely.... 25% orderers (75) {percent_customers}*100 orders (cohort_unique_customers)

In [27]:

format_dict = {'percent_customers':'{:.2%}', 'cohort_unique_customers': '{:}', 'n_unique_customers': '{:}'}
    
n_distinct_customers[:1].style.format(format_dict)

,,cohort_unique_customers,n_unique_customers,percent_customers
customer_cohort,first_order,,,
2015-01-04 00:00:00,2015-01-04 00:00:00,32,16,50.00%


In [37]:
def format_func(data):
    return "{:.0%} orderers ({:.0f})".format(data['percent_customers'], data['cohort_unique_customers'])
n_distinct_customers['formatted_percent_orders'] = n_distinct_customers.apply(format_func, axis = 1)

In [66]:
distinct_customers_pivot = n_distinct_customers.sort_index(ascending=False).reset_index()
thing = distinct_customers_pivot.pivot(index='customer_cohort', columns='first_order', values='formatted_percent_orders')

thing.head()



first_order,2015-01-04,2015-01-11,2015-01-18,2015-01-25,2015-02-01,2015-02-08,2015-02-15,2015-02-22,2015-03-01,2015-03-08,...,2015-05-10,2015-05-17,2015-05-24,2015-05-31,2015-06-07,2015-06-14,2015-06-21,2015-06-28,2015-07-05,2015-07-12
customer_cohort,,,,,,,,,,,,,,,,,,,,,
2015-01-04,50% orderers (32),12% orderers (32),9% orderers (32),3% orderers (32),3% orderers (32),3% orderers (32),NaN,3% orderers (32),NaN,3% orderers (32),...,3% orderers (32),NaN,NaN,NaN,NaN,NaN,NaN,3% orderers (32),NaN,NaN
2015-01-11,NaN,21% orderers (389),19% orderers (389),14% orderers (389),7% orderers (389),5% orderers (389),4% orderers (389),3% orderers (389),3% orderers (389),4% orderers (389),...,1% orderers (389),1% orderers (389),1% orderers (389),1% orderers (389),1% orderers (389),1% orderers (389),0% orderers (389),1% orderers (389),1% orderers (389),1% orderers (389)
2015-01-18,NaN,NaN,39% orderers (394),22% orderers (394),5% orderers (394),5% orderers (394),2% orderers (394),3% orderers (394),3% orderers (394),3% orderers (394),...,1% orderers (394),1% orderers (394),1% orderers (394),1% orderers (394),1% orderers (394),2% orderers (394),2% orderers (394),0% orderers (394),0% orderers (394),1% orderers (394)
2015-01-25,NaN,NaN,NaN,33% orderers (469),14% orderers (469),12% orderers (469),4% orderers (469),4% orderers (469),4% orderers (469),5% orderers (469),...,1% orderers (469),1% orderers (469),1% orderers (469),1% orderers (469),0% orderers (469),1% orderers (469),0% orderers (469),0% orderers (469),0% orderers (469),0% orderers (469)
2015-02-01,NaN,NaN,NaN,NaN,49% orderers (234),14% orderers (234),6% orderers (234),6% orderers (234),3% orderers (234),3% orderers (234),...,1% orderers (234),1% orderers (234),3% orderers (234),0% orderers (234),1% orderers (234),NaN,0% orderers (234),NaN,1% orderers (234),NaN


In [70]:
# thing.reset_index()
thing.reindex(sorted(thing.columns, reverse=True), axis=1).sort_values(by='customer_cohort', ascending=False)

first_order,2015-07-12,2015-07-05,2015-06-28,2015-06-21,2015-06-14,2015-06-07,2015-05-31,2015-05-24,2015-05-17,2015-05-10,...,2015-03-08,2015-03-01,2015-02-22,2015-02-15,2015-02-08,2015-02-01,2015-01-25,2015-01-18,2015-01-11,2015-01-04
customer_cohort,,,,,,,,,,,,,,,,,,,,,
2015-07-12,100% orderers (37),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-05,19% orderers (155),81% orderers (155),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-28,7% orderers (164),10% orderers (164),83% orderers (164),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-21,3% orderers (197),6% orderers (197),22% orderers (197),69% orderers (197),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-14,3% orderers (175),3% orderers (175),9% orderers (175),26% orderers (175),59% orderers (175),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-07,3% orderers (199),4% orderers (199),4% orderers (199),7% orderers (199),16% orderers (199),66% orderers (199),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-31,1% orderers (170),2% orderers (170),5% orderers (170),7% orderers (170),6% orderers (170),14% orderers (170),64% orderers (170),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-24,1% orderers (179),3% orderers (179),4% orderers (179),4% orderers (179),4% orderers (179),9% orderers (179),20% orderers (179),56% orderers (179),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-17,1% orderers (158),1% orderers (158),6% orderers (158),4% orderers (158),6% orderers (158),6% orderers (158),6% orderers (158),15% orderers (158),56% orderers (158),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
distinct_customers_pivot = n_distinct_customers.reset_index()
distinct_customers_pivot = distinct_customers_pivot.sort_values(by='customer_cohort', ascending=False)
distinct_customers_pivot.pivot(index='customer_cohort', columns='first_order', values='n_unique_customers')

first_order,2015-01-04,2015-01-11,2015-01-18,2015-01-25,2015-02-01,2015-02-08,2015-02-15,2015-02-22,2015-03-01,2015-03-08,...,2015-05-10,2015-05-17,2015-05-24,2015-05-31,2015-06-07,2015-06-14,2015-06-21,2015-06-28,2015-07-05,2015-07-12
customer_cohort,,,,,,,,,,,,,,,,,,,,,
2015-01-04,16.0,4.0,3.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2015-01-11,NaN,82.0,72.0,55.0,26.0,20.0,14.0,10.0,11.0,14.0,...,4.0,3.0,4.0,2.0,2.0,2.0,1.0,5.0,4.0,2.0
2015-01-18,NaN,NaN,152.0,85.0,21.0,21.0,7.0,10.0,10.0,11.0,...,5.0,3.0,5.0,3.0,4.0,7.0,6.0,1.0,1.0,2.0
2015-01-25,NaN,NaN,NaN,156.0,67.0,57.0,20.0,20.0,17.0,23.0,...,5.0,3.0,4.0,4.0,1.0,5.0,2.0,2.0,2.0,2.0
2015-02-01,NaN,NaN,NaN,NaN,114.0,32.0,14.0,14.0,7.0,8.0,...,3.0,2.0,7.0,1.0,3.0,NaN,1.0,NaN,2.0,NaN
2015-02-08,NaN,NaN,NaN,NaN,NaN,80.0,17.0,11.0,5.0,5.0,...,3.0,2.0,NaN,2.0,2.0,1.0,2.0,NaN,1.0,NaN
2015-02-15,NaN,NaN,NaN,NaN,NaN,NaN,66.0,15.0,7.0,13.0,...,3.0,2.0,NaN,NaN,3.0,1.0,1.0,2.0,2.0,1.0
2015-02-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,20.0,11.0,...,1.0,NaN,2.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN
2015-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,53.0,...,5.0,2.0,2.0,1.0,1.0,2.0,2.0,5.0,2.0,3.0
